## Instructions {-}

1. This is the template you may use to submit your code and report for the prediction problems on Kaggle.

2. You may modify the template if you deem fit, but it should have the information asked below.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, ParameterGrid, StratifiedKFold, train_test_split
from sklearn.linear_model import LinearRegression,LogisticRegression, LassoCV, RidgeCV, ElasticNetCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV, ParameterGrid, StratifiedKFold, RandomizedSearchCV
import itertools as it
import time as time
from catboost import CatBoostRegressor
import xgboost as xgb
from sklearn.ensemble import StackingRegressor, VotingRegressor, RandomForestRegressor, AdaBoostRegressor
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV, ElasticNetCV
from pyearth import Earth
from lightgbm import LGBMRegressor

## A.1) Data cleaning

Mention the data cleaning steps taken to prepare your data for developing the model. This may include imputing missing values, dealing with outliers, combining levels of categorical variable(s), etc.

* Put your data cleaning/preparation code with comments here
* The code should begin from reading the train data
* The code should end when you obtain the data used to develop the model in A.4

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

mputer = KNNImputer(n_neighbors=5)

# fit and transform the data
test_imputed = pd.DataFrame(imputer.fit_transform(test), columns=test.columns)
train_imputed = pd.DataFrame(imputer.fit_transform(train), columns=train.columns)

corr_matrix = train_imputed.corr().abs()

corr_y = train_imputed.corrwith(train_imputed['y'])
train_clean = train_imputed.drop(corr_y[corr_y.isna()].index.tolist(), axis = 1)
test_clean = test_imputed.drop(corr_y[corr_y.isna()].index.tolist(), axis = 1)

X = train_clean.drop(['id','y'], axis = 1)
y = train_clean.y
X_test = test_clean.drop('id', axis = 1)

scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)
X_test_scaled = scaler.transform(X_test)

## A.2) Exploratory data analysis

Mention any major insights you obtained from the data, which you used to develop the model. Please put your code or visualizations here if needed.

- Insight 1: The y variable has a skewed distribution, so logging the target variable might be necessary to improve the model.
- Insight 2: Many of the columns had null values, so I imputed the missing values using KNN=5
- Insight 3: Some of the columns had null correlation with the target variable, so I removed these columns

## A.3) Feature selection/reduction

Mention the steps for feature selection/reduction. Please put your code or visualizations here if needed.

- Step 1: I ran an initial CatBoost model on the scaled data and arranged the features in descending order of importance.
- Step 2: I removed all the features that had an importance of 0. There were 659 predictors remaining, and I used these features for the CatBoost model in my ensemble.
- Step 3: I ran a MARS model (degree=1) using the 659 predictors from the CatBoost feature importance.
- Step 4: I removed all the features from the MARS model that had an importance of 0. There were 26 predictors remaining, which I used for the MARS and LGB models in my ensemble. 

In [ ]:
# Developing model 1: CatBoost
# initial catboost model
catboost = CatBoostRegressor(subsample=0.5,reg_lambda=0,num_leaves=31,n_estimators=1000,max_depth=6,learning_rate=0.05)
catboost.fit(X_scaled, np.log(y))

# feature selection
predictor = pd.Series(X.columns, name = 'predictor')
rel_importance = pd.Series(catboost.feature_importances_, name = 'importance')
importance = pd.concat([predictor, rel_importance], axis = 1).sort_values(by = 'importance', ascending = False)
catboost_features = importance['predictor'].loc[importance['importance'] > 0].tolist()
X_catboost = X[catboost_features]
test_catboost = test_clean[X_catboost.columns]

In [ ]:
# Developing model 2 and 3: MARS and LGBM
# initial mars model
mars = Earth(max_terms = 500, max_degree = 1, feature_importance_type = 'rss')
mars.fit(X_catboost, np.log(y))

# feature selection
predictor = pd.Series(X.columns, name = 'predictor')
rel_importance = pd.Series(mars.feature_importances_, name = 'importance')
importance = pd.concat([predictor, rel_importance], axis = 1).sort_values(by = 'importance', ascending = False)
mars_pred = importance['predictor'].loc[importance['importance'] > 0].tolist()
X_mars = X[mars_pred]
test_mars = test_clean[X_mars.columns]

## A.4) Developing the model

Mention the logical sequence of steps taken to obtain the final model. 

- Model 1: CatBoostRegressor on 659 predictors with the params: subsample=0.5, reg_lambda=0, num_leaves=31, n_estimators=1000, max_depth=6, learning_rate=0.05
- Model 2: MARS on 26 predictors with the params: degree=1, max_terms=500
- Model 3: LGBM on 26 predictors with the params: subsmaple=0.75, reg_lambda=0, reg_alpha=0, num_leaves=31, n_estimators=1000, max_depth=4, learning_rate=0.01, colsample_bytree=0.5

In [ ]:
# Model 1
catboost_final = CatBoostRegressor(subsample=0.5, reg_lambda=0, num_leaves=31, n_estimators=1000,
                                   max_depth=6, learning_rate=0.05).fit(X_catboost, np.log(y))

# Model 2
mars_final = Earth(max_terms = 500, max_degree = 1).fit(X_mars, np.log(y))

# Model 3
lgbm_mars = LGBMRegressor(subsample=0.75, reg_lambda=0, reg_alpha=0, num_leaves=31, n_estimators=1000,
                     max_depth=4, learning_rate=0.01, colsample_bytree=0.5).fit(X_mars,np.log(y))

Step taken to obtain my final model and RMSE: 
- Step 1: I ensembled these three models using StackingRegressor with ElasticNetCV() as the metamodel. 
- Step 2: I  multiplied the predicted values by 1.27 and subtracted these values from the logged `y` values in the training dataset to find the residuals
- Step 3: I fit the same ensembled models on the residuals. 
- Step 4: I added the predicted residuals to the predicted `y` values
- Step 5: I calculated the exponential value of the final predicted values and multiplied it by 1.27

In [ ]:
en = StackingRegressor(estimators = [('catboost_final', catboost_final),('lgbm_mars', lgbm_mars),('mars_final', mars_final)],
                       final_estimator=ElasticNetCV(),                                          
                       cv = KFold(n_splits = 5, shuffle = True, random_state=1))
en.fit(X_scaled,np.log(y))

In [ ]:
# calculating residuals on train data 
res = np.log(y) - en.predict(X_scaled)
# realized there was a discrepancy between the logged y train values and teh predicted y train values 

# fitting ensemble on residuals
en_res = StackingRegressor(estimators = [('catboost_final', catboost_final),('lgbm', lgbm_mars),('mars_final', mars_final)],
                       final_estimator=ElasticNetCV(),                                          
                       cv = KFold(n_splits = 5, shuffle = True, random_state=1))
en_res.fit(X_scaled,res)

In [ ]:
pred = pd.DataFrame()
pred['id'] = test['id']
pred['y'] = np.exp(en.predict(X_test_scaled)+en_res.predict(X_test_scaled))*1.27
pred = pred.set_index(['id'])
pred.to_csv('Witarsa_Ashley.csv')

## A.5) Discussion

Please provide details of the models/approaches you attempted but encountered challenges or unfavorable outcomes. If feasible, kindly explain the reasons behind their ineffectiveness or lack of success. Additionally, highlight the significant challenges or issues you encountered during the process.

**My initial approach to feature selection was to remove predictors that were very highly correlated with each other before running feature selection using MARS. If two predictors had a correlation above 0.9, I would drop one of the predictors. I would also drop predictors that had a very low correlation with the target variable `y`. However, by using this approach, I was not able to get my RMSE below 9.0 despite ensembling several models such as CatBoost and XGBoost. It is possible that this method was ineffective because removing these predictors might have resulted in underfitting and prevented my models from being able to capture the subtle but important effects of these predictors on the target variable.**

**In terms of the models I attempted, I initially found that using XGBoost improved my RMSE score significantly compared to other models such as MARS and Random Forest. However, my RMSE was still above the target of 8.75. I found that by ensembling XGBoost and CatBoost and selecting predictors based on MARS feature selection, I was able to get my RMSE to around 8.77. Nevertheless, I realized that my model might still be underfitting, since MARS feature selection cut down the number of predictors that had non-zero importance to around 20-30 predictors.**

**I decided to perform feature selection using CatBoost as well. I tuned and trained a CatBoost model using all the predictors, and chose the predictors that had a non-zero importance. I then ran another CatBoost model using the new set of predictors. I also ran a MARS model on the set of predictors I found from the CatBoost feature selection. I used this MARS model to conduct another feature selection. I tried to stack the CatBoost and MARS models together using various metamodels and found that ElasticNetCV worked best. Adding an LGB model to my ensemble with the MARS feature selection also improved my RMSE. By using predictors from two different feature selections, one with many predictors and one with very few predictors, I was able to prevent the underfitting problem I had initially. This model allowed me to get to an RMSE of below 8.75.**

**Through trial and error, I realized that multiplying my predicted values by a multiplier slightly improved my RMSE. I tried multiplyng my predicted values by different multipliers ranging from 1.23-1.28. It seemed that multiplying my predicted values by 1.27 was best, and I was able to achieve a final RMSE of 8.41. However, I still was not able to get the desired RMSE of under 8.4 on the public leaderboard, which is the benchmark used in Stat 303-3 for the final waiver.**

**To further improve my model, I decided to further analyze my predicted values. By using my ensemble model on the training set, I realized that there was a discrepancy between the predicted train values and the actual y train values. I decided to fit and train the same ensemble model on the residuals, and I was able to achieve a final RMSE of around 8.39 on the public leaderboard.**

## A.6) Conclusion

* Do you feel that you gain valuable experience, skills, and/or knowledge? If yes, please explain what they were. If no, please explain.
* What are things you liked/disliked about the project and/or work on the project?

**Yes, I feel like I gained valuable experience and skills in terms of feature selection as well as creating and ensembling models. I previously did not have any experience with models like CatBoost and Light Gradient Boosting, and I also had very limited experience with ensembling models. I am currently also taking Stat 303-3, so doing this project has been helpful in allowing me to apply other models that I have learned in that class such as MARS, XGBoost, and Random Forests. Although I did not end up using all the models that I explored, each model that I utilized allowed me to explore the dataset in more depth.**

**I enjoyed being able to use the various modeling tools that I have learned this quarter. It was exciting to try combinations of different models to see which ones would work best. However, I did encounter several struggles throughout this project. Tuning some of the models and ensembling took quite long, and it was a little frustrating when the models I explored did not improve my RMSE.** 

**Overall, despite the challenges I faced during this project, I believe it has been a valuable learning experience. It has not only enhanced my coding and modeling skills but also taught me the importance of persistence and creativity when it comes to solving problems. I am now more confident in my ability to tackle real-world datasets and apply a variety of modeling techniques to achieve better results. I am grateful for the opportunity to explore various models and to experiment with ensembling techniques. This project has definitely broadened my understanding of feature selection, model ensembling, and the overall process of building effective predictive models. I look forward to applying these skills to future projects and further advancing my expertise in machine learning and data science.**

## Please make sure your github repo has all the code and  ensure that your code is capable of reproducing the outcomes you have submitted. It is important to avoid any form of academic misconduct or cheating by using your peer's submission file